In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
import time
import datetime

from keras import layers, losses, utils, optimizers, callbacks
from keras.applications import VGG16
from tqdm import tqdm
from base64 import b64encode

### Get KITTI dataset

In [ ]:
# training data
TRAIN_DATA_DIR = "/input/kittiroadsegmentation/training/image_2/"

# ground trouth segmentation
TRAIN_GT_DIR = "/input/kittiroadsegmentation/training/gt_image_2/"

# test data
TEST_DATA_DIR = "/input/kittiroadsegmentation/testing/"

# get size of training samples
TRAINSET_SIZE = int(len(os.listdir(TRAIN_DATA_DIR)) * 0.8)
VALIDATION_SIZE = int(len(os.listdir(TRAIN_DATA_DIR)) * 0.1)
TESTSET_SIZE = int(len(os.listdir(TRAIN_DATA_DIR)) - TRAINSET_SIZE - VALIDATION_SIZE)

### Constants

In [ ]:
IMG_SIZE = 128
N_CHANNELS = 3
N_CLASSES = 1
SEED = 123

### Dataset preprocessing

#### Parse data into masks and images as a dict, generation of dataset splits

In [ ]:
def ParseImages(image_path: str) -> dict:
    # read standard road images and decoding
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)

    # three types of image paths: (um_road, umm_road, uu_road)
    mask_path = tf.strings.regex_replace(image_path, "image_2", "gt_image_2")
    mask_path = tf.strings.regex_replace(mask_path, "um_", "um_road_")
    mask_path = tf.strings.regex_replace(mask_path, "umm_", "umm_road_")
    mask_path = tf.strings.regex_replace(mask_path, "uu_", "uu_road_")

    # read and decode masks
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)

    # labeling
    non_road_label = np.array([255, 0, 0])
    road_label = np.array([255, 0, 255])
    other_road_label = np.array([0, 0, 0])

    # convert mask to binary
    mask = tf.experimental.numpy.all(mask == road_label, axis=2)
    mask = tf.cast(mask, tf.uint8)
    mask = tf.expand_dims(mask, axis=-1)

    return { "image": image, "segmentation_mask": mask}

# generate dataset splits: test, train, val
dataset = tf.data.Dataset.list_files(TRAIN_DATA_DIR + "*.png", seed=SEED)
dataset = dataset.map(ParseImages)

# splitting
train_ds = dataset.take(TRAINSET_SIZE + VALIDATION_SIZE)
validation_ds = train_ds.skip(TRAINSET_SIZE)
train_ds = train_ds.take(TRAINSET_SIZE)
test_ds = dataset.skip(TRAINSET_SIZE - VALIDATION_SIZE)

### Normalizing and Data Augmentation

In [ ]:
@tf.function
def Normalize(input_image, input_mask) -> tuple:
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image, input_mask

@tf.function
def LoadTrainImages(datapoint) -> tuple:
    # resize images and masks
    input_image = tf.image.resize(datapoint["image"], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint["segmentation_mask"], (IMG_SIZE, IMG_SIZE))

    # data augmentation by randomly flipping the image and generate new training data
    if tf.random.uniform() > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    # normalize images
    input_image, input_mask = Normalize(input_image, input_mask)

    return input_image, input_mask

@tf.function
def load_image_test(datapoint: dict) -> tuple:
    # resizing
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

    # normalizing
    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

